In [ ]:
from pycalphad import Database, equilibrium, variables as v
# from pycalphad.core.utils import filter_phases
import random
import numpy as np
import time
from tqdm import tqdm
# from tqdm.auto import tqdm

# constants:
P = 101325
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)
# TODO: check 'PM' param
all_components = ['LI', 'BE', 'NA', 'MG', 'AL', 'K', 'CA', 'SC', 'TI', 'V', 'CR', 'MN', 'FE',
                  'CO', 'NI', 'CU', 'ZN', 'GA', 'Y', 'ZR', 'NB', 'MO', 'TC',
                  'RH', 'PD', 'AG', 'CD', 'IN', 'SN', 'BA', 'LA', 'CE', 'PR', 'ND', 'SM', 'EU',
                  'GD', 'TB', 'DY', 'HO', 'ER', 'HF', 'TA', 'W', 'RE', 'OS', 'IR',
                  'PT', 'AU', 'HG', 'TL', 'PB', 'BI', 'C', 'SI']
all_phases = list(db.phases)
all_phases.sort()
all_components.sort()

In [108]:
from numpy import save

save('./DataSets/all_phases.npy', all_phases)

In [109]:
import os
from modules.GetGm import getGM


def getTimeGM(db, Components, all_phases, possible_phases, amounts,T):
    tic = time.perf_counter()
    gm = getGM(db, Components, all_phases, possible_phases, amounts,T)
    toc = time.perf_counter()
    return (toc-tic, gm)

def getPhases(int_mask, all_phases=all_phases):
    return [all_phases[i] for i in range(len(all_phases)) if (int(int_mask) & (1 << i)) > 0]

def phasesToMask(phases, all_phases=all_phases):
    phases.sort()
    mask = [1 if phase in phases else 0 for phase in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)


def phaseToMask(phase, all_phases=all_phases):
    mask = [1 if phase == phase_ else 0 for phase_ in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)

def compToMask(components, all_components=all_components):
    components.sort()
    mask = [1 if comp in components else 0 for comp in all_components][::-1]
    return int(''.join(map(str, mask)), 2)

def getComponents(int_mask, all_components=all_components):
    return [all_components[i] for i in range(len(all_components)) if (int_mask & (1 << i)) > 0]

def parseArray(amounts_str):
    amounts = amounts_str.replace('[', '').replace(']', '')
    return np.array(amounts.split(','), dtype=np.float32)

def addDFLineToCSV(df, path='./DataSets/parsedDataset.csv'):
    df.to_csv(path, mode='a', sep=";", header=not os.path.exists(path))


In [110]:
import pandas as pd

df_origin = pd.read_csv('DataSets/main_test.csv',delimiter = ';')
df_origin = df_origin[df_origin['Error'].isna()].reset_index(drop=True)
df = df_origin.reset_index(drop=True)
df_origin['gm_time'] = np.nan
df_origin['GM'] = np.nan
df_origin['GM'] = df_origin['GM'].astype(object)
df = df.drop(['Error'], axis =1)
df = df.rename(columns={"T": "Temp"})
df['Components'] = df['Components'].apply(getComponents)
df['possible_phases'] = df['possible_phases'].apply(getPhases)
df['amounts'] = df['amounts'].apply(parseArray)
# print(df_origin.dtypes)
# print(df.dtypes)
df

,iter,Temp,amounts,Components,phases,ellapsed_time,possible_phases,P
0,500,1991,"[0.106400184, 0.070539184, 0.09098721, 0.08310...","[CO, CR, MO, NI, TA, V, W, ZR]",6084483977492414823814226985991697806408637340...,28.562051,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
1,501,1613,"[0.055818617, 0.064889625, 0.21049123, 0.10578...","[CO, CR, MO, NI, TA, V, W, ZR]",0,25.476817,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
2,502,1253,"[0.0770591, 0.054417454, 0.11515643, 0.1760729...","[CO, CR, MO, NI, TA, V, W, ZR]",6813845498726864658394108293986205049292797343...,25.641211,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
3,503,429,"[0.1490746, 0.3030639, 0.28587595, 0.050671652...","[CO, CR, MO, NI, TA, V, W, ZR]",3642361157803544944038421687014756117706903781...,24.184063,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
4,504,1324,"[0.29517522, 0.29847562, 0.06297051, 0.1217284...","[CO, CR, MO, NI, TA, V, W, ZR]",6748105731502757109685565502551083482578950923...,24.452664,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
...,...,...,...,...,...,...,...,...
735,1233,993,"[0.3217174, 0.22605425, 0.19374935, 0.05437803...","[CO, CR, MO, NI, TA, V, W, ZR]",6748105731502757109685565502551083482578950923...,21.828628,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
736,1234,566,"[0.10788198, 0.27838793, 0.09425418, 0.1218519...","[CO, CR, MO, NI, TA, V, W, ZR]",6748105731502757109685565502551083482578950923...,21.744828,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
737,1235,1976,"[0.10189047, 0.27404854, 0.3013579, 0.05023442...","[CO, CR, MO, NI, TA, V, W, ZR]",6084483977492414823814226985991697806408637340...,22.094840,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
738,1236,652,"[0.21985678, 0.18890162, 0.08386024, 0.0733089...","[CO, CR, MO, NI, TA, V, W, ZR]",6748105731502757109685565502551083482578950923...,22.048363,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325


In [ ]:
i = df_origin.index[0]

for i in tqdm(df_origin.index):
    if i < 69:
        continue
    df_line = df_origin.loc[i:i].copy()
    df_line.loc[:,'gm_temp'] = df.loc[i:i].apply(lambda x: getTimeGM(db, x.Components, all_phases, x.possible_phases, x.amounts, x.Temp), axis=1)
    df_line.loc[:,'gm_time'] = df_line.apply(lambda x: x.gm_temp[0], axis=1)
    df_line.loc[:,'GM'] = df_line.apply(lambda x: x.gm_temp[1], axis=1)
    df_line = df_line.drop(['gm_temp'], axis=1)
    addDFLineToCSV(df_line)

In [106]:
np.array(df_line['GM'][i])[~np.isnan(df_line['GM'][i])]

array([ -50812.70323749,  -36789.23996588,  -45411.04141155,
        -33344.3475635 ,  -33344.3475635 ,  -38147.73493824,
       -159689.40032381,  -34767.42453429,  -40583.57088381,
        -38396.41949325])